In [1]:

# for sentence play in Anki
# initial date: 2015-11-20

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs
import re

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\mae\\sentencePlayAnki.txt', 'w', 'utf-8')

# read list of words with images [captions] into imglist
with codecs.open('E:\\1Now\\taglist\\mwlDailyImg20141124_20151205.txt', 'r', 'utf-8') as f:
    imglist = f.read().splitlines() # strip '\n'
#     f.close() # not necessary with with statement
imginhw = [] # track which images match head words

# tag word lists
# core3, tG, awl are lists core31, tGtuned, 569awl excluced from 2939exclu
tfiles = ['acaWriting.txt', 'airTravel.txt', 'bodyPart.txt', 'campus.txt', 'economics.txt', 'emotion.txt', 
          'employment.txt', 'environment.txt', 'family.txt', 'health.txt', 'job.txt', 'legal.txt', 
          'nonCount.txt', 'personality.txt', 'sports.txt', 'timeAdv.txt', 'timePeriod.txt', 'weather.txt',
          'core3.txt', 'tG.txt', 'awl.txt']

topics = []
for tfile in tfiles:
    topic = {}
    with codecs.open('E:\\1Now\\taglist\\' + tfile, 'r', 'utf-8') as f:
        twords = f.read().splitlines()
        f.close()
    topic['words'] = twords
    topic['topic'] = tfile.split('.')[0]
    topics.append(topic)
    

# read TOEFL/GRE word list
# with codecs.open('E:\\1Now\\mae\\tginmwaled.txt', 'r', 'utf-8') as f:
#     tgwl = f.read().splitlines() # strip '\n'
#     f.close()

#results = geweiDict.find({"fr":"y"}, {"_id":0, "hw":1, "infl":1, "subhw":1, "prn":1, "alles":1, 
#                              "idpvs":1, "sublist":1, "audiofile":1, "alles":1})
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1, "prn":1, "alles":1, "img":1,
                              "idpvs":1, "sublist":1, "homog":1, "alles":1, "tag":1, "label":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)

# total number of head words
totalhw = 0

# threshold of the length of definition
defthres = 12

# Produce Anki note fields and tags:
for result in results:
    
    word = result.get("hw")    
    if word:
        #print word
        totalhw += 1
    else:
        print "no head word"
        continue

    
    # get items of result
    tagindb = result.get('tag')
    label = result.get('label')
    
    infls = result.get("infl")
    subhws = result.get("subhw")   
    imgindb = result.get("img")
    
#     homograph = 0
    homog = result.get("homog")
    
    idpvs = result.get("idpvs")
    sublists = result.get("sublist") #sublists is a list whose items are dictionary
    
    if sublists:
        cnt = 1
        idiom = 0
        
        # each "es" or "idpv" of sublist in sublists will be an Anki note
        for sublist in sublists: #sublist is dictionary        
            
            # 9 fields with tags: PlaySent,def,Wow,prn,hw,audio,footnote,img,caption   tags
            
            # Anki note field #1: PlaySent (incomplete example sentences for cloze test)
            # PlaySent must be the first field because codes for it include continue statement which
            # may not create a note for this loop.
            if sublist.has_key("es"):                
                # no example sentence but have definition, write head word to PlaySent field
                if len(sublist["es"]) == 0 and len(sublist["defi"]) >= defthres:
                    #print defthres, " --- ", word                    
                    fdict.write(word)
                # this es is empty and "def" is not a definition, discard this fake definition
                elif len(sublist["es"]) == 0 and len(sublist["defi"]) < defthres:
                    #print word, ": empyt example and not a defintion"
                    continue                    
                else:
                    for sPlay in sublist["es"]:                    
                        #sPlay = es.encode('ascii', 'ignore').replace(word, "___")                        
                        #sPlay = es
                        insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                        sPlay = insensitive_replace.sub('___', sPlay)
                        
                        if infls:
                            for infl in infls:
                                sPlay = sPlay.replace(infl, "...")
                        if subhws:
                            for subhw in subhws:
                                sPlay = sPlay.replace(subhw, "???")
                        fdict.write(str(cnt) + '<font color="blue">-es: </font>' + sPlay + "<br>")
                    
            if sublist.has_key("idpv"):
                if len(sublist["idpv"]) == 0:
                    #print word, ": empyt idpv"
                    continue
                #sPlay = sublist["idpv"].encode('ascii', 'ignore').replace(word, "___")
                #sPlay = sublist["idpv"].encode('ascii', 'ignore')
                sPlay = sublist["idpv"]
                sPlay = re.sub(r'\s+(?=\d)', '<br><br>', sPlay) # replace the space before a number with '<br><br>'
                
                insensitive_replace = re.compile(re.escape(word), re.IGNORECASE)
                sPlay = insensitive_replace.sub('___', sPlay)
                if sPlay.find(":") >= 0: idiom = 1
                
                if infls:
                    for infl in infls:
                        sPlay = sPlay.replace(infl, "...")
                if subhws:
                    for subhw in subhws:
                        sPlay = sPlay.replace(subhw, "???")
                fdict.write('<font color="blue">*idpv*</font> ' + sPlay + "<br>")
            #if cnt%1 == 0: fdict.write('<br>')
            fdict.write(" \t")               
            

            # Anki note field: definition
            if sublist.has_key("defi"):                
                #fdict.write(str(cnt) + '-def: ' + sublist["def"].encode('ascii', 'ignore') + "<br><br>")
                fdict.write(str(cnt) + '-def: ' + sublist["defi"] + "<br><br>")            
            fdict.write(" \t")
                
            # Anki note field: Wow (complete example sentences)
            if sublist.has_key("es"):  
                for es in sublist["es"]:
                    fdict.write(str(cnt) + '<font color="red">-es: </font>' + 
                                es + "<br>")                
                                #es.encode('ascii', 'ignore') + "<br>")                
            if sublist.has_key("idpv"):                
                fdict.write('<font color="red">*idpv*</font> ' + 
                            re.sub(r'\s+(?=\d)', '<br><br>', sublist["idpv"]) + "<br>")
                            #sublist["idpv"].encode('ascii', 'ignore') + "<br>")
            #if cnt%1 == 0: fdict.write('<br>') 
            fdict.write(" \t")
            
       
            
            # Anki note field: Pronunciation in ASCII IPA symbols
            prns = result.get("prn")
            if prns:
                for prn in prns:
                    fdict.write("/" + prn + "/<br>")
            fdict.write(" \t")  # add one space before tab to separate tabs in case a field is empty.     
            
            # Anki note field: head word
            fdict.write(word + "\t")     
            
            
            # Anki note field: Audio    
            if homog:
                fdict.write("[sound:" + homog + ".wav]")
#                 homograph = 1                
            elif word == 'con':                    
                # con is a key word of Windows, a filename as 'con.wav' is not allowed, so use 'con1.wav'                
#                 print "---------con-----------"
                fdict.write("[sound:con1.wav]")                   
            else: 
                fdict.write("[sound:" + word + ".wav]")
                    
            if subhws:
                for subhw in subhws:
                    if subhw != word: fdict.write("<br>[sound:" + subhw + ".wav]")
            fdict.write(" \t")    

            
            # make tags 
            tags = ''
            if tagindb:
                for tag in tagindb:
                    tags += ' ' + tag
            for topic in topics:
                if word in topic['words']: tags += ' ' + topic['topic']
                    
            if label:
                if 'slang' in label: tags += ' slang'
                if 'baseball' in label: tags += ' baseball'
                if 'sports' in label: tags += ' sports+' # beyond core sports.txt
                
            if idiom == 1: tags += " idiom"

            
            # Anki note field: Footnote            
            fdict.write("<br><br>{" + tags + "}<br><br>\t")
            
            # Anki note fields: Image and Caption
            img = ' '
            caption = ' '
            imgflag = 0
            imgcnt = 0           
            # check if hw has images in db (mwl book)
            if imgindb:
                for x in imgindb:
                    img += '<img src="' + x + '" />'
                
            # check if hw has images in mwl daily images
            for imgname in imglist:
                if word == imgname.split('.')[0]: 
                    imginhw.append(imgcnt) # record the index of items in imglist that match hw
                    imgl = imgname.split(';')
                    #print imgl
                    img += '<img src="' + imgl[0] + '" />'
                    imgflag = 1
                    if len(imgl) == 2: caption += imgl[1]
                    elif len(imgl) == 3: caption += imgl[1] + ';' + imgl[2]
                
                imgcnt += 1
                 
            fdict.write(img + "\t")
            fdict.write(caption + "\t")

            # Anki tags
            if imgflag == 1: tags += " imgmwlD" # images from mwl daily images
            fdict.write(tags + "\n")
            
            cnt += 1
    
fdict.close()
print "total head words: ", totalhw


# fd = codecs.open('E:\\1Now\\mae\\imgNameInHw.txt', 'w', 'utf-8')
# for i in range(0, len(imglist)):
#     if i not in imginhw: print imglist[i]
#     else: fd.write(imglist[i] + '\n')
# fd.close()  
    




total head words:  39195


In [14]:
# create an Anki import file with only hw not empty for a word list
# 
import codecs

fout = codecs.open('E:\\1Now\\mae\\3fields1tag4sPlayAnki.txt', 'w', 'utf-8')

with codecs.open('E:\\1Now\\mae\\tgnotinmwaled.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()
cnt = 0
for line in lines:
    if not line: continue
    # 9 fields with tags: PlaySent,def,Wow,prn,hw,audio,footnote,img,caption   tags
    # first field must not be empty, so use hw to hold the position
    # PlaySent, hw, audio, tags not empty
    anote = line + '\t \t \t \t' + line+'\t' + '[sound:'+line+'.wav]\t' + ' \t \t \tnotinmwaled tg'
    #print anote
    fout.write(anote + '\n')
    cnt += 1
print cnt, ' words'    
fout.close()    
    

164  words


In [ ]:
# 2015-11-24
# 
# creat an Anki import file for a list of words

# to-do: add tags for differet group words


import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511


fdict = codecs.open('anki\\soundPlayAnki.txt', 'w', 'utf-8')
with codecs.open('anki\\tmpsoundlist.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
    f.close()
    

# Produce Anki note fields and tags:
for line in lines:
    wordprn = line.split(';')
    word = wordprn[0]
    print word
    
    # get prn from mongodb if word is a head word of the collection
    prnfd = ' '
    prnflag = 0
    
    #results = geweiDict.find({"hw": word}, {"_id":0, "hw":1, "prn":1})
    results = geweiDict.find({"$or": [{"hw": word}, {"subhw": word}]}, 
                             {"_id":0, "hw":1, "subhw":1, "prn":1})
    for result in results:
        hw = result.get("hw")
        subhws = result.get('subhw')
        print hw, '---', word       
        if hw != word and word not in subhws: continue
        
        prns = result.get("prn")
        print prns
        if prns:
            prnflag = 1
            for prn in prns:
                prnfd += "/" + prn + "/ "
    
    # Anki note field: head word
    fdict.write(word+'\t')
    
    # Anki note field: Audio
    fdict.write("[sound:" + word + ".wav]")
    fdict.write(" \t")
    
    # Anki note field: Pronunciation in ASCII IPA symbols
    if prnflag == 0 and len(wordprn) >= 2: #like acropolis;/.'krA:p.l.s/
        prnfd += wordprn[1]
            
    fdict.write(prnfd + '\t')  
    
    #Anki note field: footnote (about pronunciation, or an image link)
    #like: acropolis;/.'krA:p.l.s/;<img src="acropolis.jpg" /> or zillion; ;(zill-ion, /'z!l   j.n/) (two syllables)   
    if len(wordprn) == 3: fdict.write(wordprn[2])
    fdict.write("\n")

fdict.close()    

In [ ]:
# this snippets of codes organize a raw list to a unique sorted list
# first add new items in raw text, the snippets will create a unique sorted list text

import codecs

fafter = codecs.open('anki\\tmpsoundlist.txt', 'w', 'utf-8')

with codecs.open('E:\\1Now\\taglist\\chocolateword4Anki.txt', 'r', 'utf-8') as fraw:
    lines = fraw.read().splitlines()
    fraw.close()

cnt = 0    
#ulists = sorted(set(lines))
#case insensitive sorting
ulists = sorted(set(lines), key=lambda s: s.lower())
#ulists = sorted(lines)

for item in ulists:    
    fafter.write(item + '\n')
    cnt += 1
    print cnt, ': ', item
fafter.close()